<center><img src="../images/logo.png" alt="Your Image" style="width: 500px; height: auto"></center>

# Approx. valuation of an at-the-money call option via a Kolmogorov-Arnold Network


In this notebook, we show based on the example of an approximate formula for pricing an at-the-money call option how a Kolmogorov-Arnold Network (KAN) can be used for the approximation and fast evaluation of classical valuation methods.

For basics about the pricing of a call option with the Black-Scholes-Merton model, see here: [European Call Option](Time_Series_Predicition/Predicting_Stock_Prices_ESN.ipynb). For the at-the-money call option, we have $S = K e^{-r(T-t)}$, such that plugging this into the standard Black-Scholes formula gets us:

$C(S,t) = N(0.5 \sigma \sqrt{T-t}) -  N(-0.5 \sigma \sqrt{T-t})$. 

Taylor approximation implies for small $x$, that

$N(x) = N(0) + N'(0) x + \mathcal{O}(x^2) $,

such that with $N'(0) = 1/\sqrt{2 \pi}$ (and $r = 0$) the appoximation of the price $C$ of a at-the-money call with maturity $T$ and volatility $\sigma$ is given by the equation:

$C(S,t) \approx 0.4 S \sigma \sqrt{T-t} $.



What makes KANs special is, that they have learnable activation functions on edges (“weights”) and no linear weights at all, meaning every
weight parameter is replaced by a univariate function parametrized as a spline. For more details about the relevant principles of Kolmogorov-Arnold Networks, please refer to https://arxiv.org/pdf/2404.19756. Note, that the implementation and application to our use case here is based on https://github.com/KindXiaoming/pykan?tab=readme-ov-file.


## Import

In [ ]:
# import statements
import math
from numbers import Number
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

import torch
torch.set_default_dtype(torch.float64)
from kan import *


%matplotlib inline

## Generate and plot data

Given data points (T,sigma,S,call_price), we are interested in figuring out the symbolic formula. 

We create input data for the training of the neural network, including
- time to maturity (T)
- volatility (sigma)
- spot (S)

In [ ]:
def approx_ATM_euro_vanilla_call(S_times_sigma, T):
    """ 
    Compute an approximation of an at-the-money European call price
    """
    call = 0.4*S_times_sigma*torch.sqrt(T)
    return call


def create_dataset(f, 
                   n_var=2, 
                   f_mode = 'col',
                   train_num=1000, 
                   test_num=1000,
                   normalize_input=False,
                   normalize_label=False,
                   device='cpu',
                   seed=42):

    torch.manual_seed(seed)
    
    train_input = torch.zeros(train_num, n_var)
    test_input = torch.zeros(test_num, n_var)
    
    # time to maturity
    train_input[:,0] = torch.rand(train_num,)*(1.-0.1)+0.1
    test_input[:,0] = torch.rand(test_num,)*(1.-0.1)+0.1
    # vol*spot
    train_input[:,1] = (torch.rand(train_num,)*(0.3-0.05)+0.05)*(torch.rand(train_num,)*(1.1-0.9)+0.9)
    test_input[:,1] = (torch.rand(train_num,)*(0.3-0.05)+0.05)*(torch.rand(train_num,)*(1.1-0.9)+0.9)

    if f_mode == 'col':
        train_label = f(train_input)
        test_label = f(test_input)
    elif f_mode == 'row':
        train_label = f(train_input.T)
        test_label = f(test_input.T)
    else:
        print(f'f_mode {f_mode} not recognized')
        
    # if has only 1 dimension
    if len(train_label.shape) == 1:
        train_label = train_label.unsqueeze(dim=1)
        test_label = test_label.unsqueeze(dim=1)
        
    def normalize(data, mean, std):
            return (data-mean)/std
            
    if normalize_input == True:
        mean_input = torch.mean(train_input, dim=0, keepdim=True)
        std_input = torch.std(train_input, dim=0, keepdim=True)
        train_input = normalize(train_input, mean_input, std_input)
        test_input = normalize(test_input, mean_input, std_input)
        
    if normalize_label == True:
        mean_label = torch.mean(train_label, dim=0, keepdim=True)
        std_label = torch.std(train_label, dim=0, keepdim=True)
        train_label = normalize(train_label, mean_label, std_label)
        test_label = normalize(test_label, mean_label, std_label)

    dataset = {}
    dataset['train_input'] = train_input.to(device)
    dataset['test_input'] = test_input.to(device)

    dataset['train_label'] = train_label.to(device)
    dataset['test_label'] = test_label.to(device)

    return dataset



In [ ]:
f = lambda x: approx_ATM_euro_vanilla_call(x[:,[1]],x[:,[0]])
dataset = create_dataset(f)
dataset['train_input'].shape, dataset['train_label'].shape

In [ ]:
plt.figure(figsize=(16,4))

plt.subplot(1,3,2)
plt.hist(dataset['train_input'][:,1], bins=20, alpha=0.5, label='training')
plt.hist(dataset['test_input'][:,1], bins=20, alpha=0.5, label='test')
plt.xlabel('S*vol')
plt.ylabel('#points')
plt.legend()

plt.subplot(1,3,3)
plt.hist(dataset['train_input'][:,0], bins=20, alpha=0.5, label='training')
plt.hist(dataset['test_input'][:,0], bins=20, alpha=0.5, label='test')
plt.xlabel('time to maturity')
plt.ylabel('#points');
plt.legend()

## Creating and Training the KAN

In this section, we build and train the KAN.


### Training with sparsification.

 Starting from a fully-connected [2,5,1] KAN (i.e., the function includes 2D inputs, 1D output, and 5 hidden neurons).


In [ ]:
torch.set_default_dtype(torch.float64)
model = KAN(width=[2,5,1], grid=10, k=3, seed=42)

The initialization of the model can be vizualized by calling the plot method on the model.
 

In [ ]:
model(dataset['train_input']);
model.plot()

### Training the Model

In this section, we train the KAN.
Note: small bug fix for pykan 0.2.3: https://github.com/KindXiaoming/pykan/issues/375




In [ ]:
model.fit(dataset, opt="LBFGS", steps=20)

Plot the trained model.

In [ ]:
model.plot()

In [ ]:
model = model.prune()
model(dataset['train_input'])
model.plot()

In [ ]:
model.fit(dataset, opt="LBFGS", steps=50);


In [ ]:
model.plot()

In [ ]:
model.fit(dataset, opt="LBFGS", steps=200);


In [ ]:
model.auto_symbolic()

In [ ]:
model.fit(dataset, opt="LBFGS", steps=200);

### Obtain the symbolic formula

In [ ]:
formula = model.symbolic_formula()[0][0]
ex_round(formula,2)